### Load common functions, libs and vars

In [7]:
# %load common.py
import iris
import xarray
import os
from numcodecs import Blosc
import s3fs
import zarr
import intake
from datetime import datetime, timezone, timedelta
import cf_units
import json
import  dask_kubernetes
import distributed
import boto3
from iris.experimental.equalise_cubes import equalise_attributes
import pandas as pd
import sys
import numpy as np

sys.path.append(os.path.normpath(os.getcwd()))
from offsetmap import OffSetS3Map

sqs = boto3.client('sqs')

AWS_EARTH_TIME_FORMAT = '%Y-%m-%dT%H:%M:%SZ'
# SQS_QUEUE_URL = 'https://sqs.eu-west-2.amazonaws.com/536099501702/aws-earth-test'
SQS_QUEUE_URL = 'https://sqs.eu-west-2.amazonaws.com/536099501702/rolling_zarr_test_queue'

BUCKET = "metoffice-aws-earth-zarr"

def del_msg(msg):
    sqs.delete_message(
        QueueUrl=SQS_QUEUE_URL,
        ReceiptHandle=msg['receipt_handle']
    )
    
def get_messages(max_num=10):
    res = sqs.receive_message(QueueUrl=SQS_QUEUE_URL, MaxNumberOfMessages=max_num, VisibilityTimeout=60*10)
    messages  = []
    for message in res['Messages']:
        msg = json.loads(message['Body'])
        msg['receipt_handle'] = message['ReceiptHandle']
        messages.append(msg)
    return messages

def get_zar_path(meta):
    base = f"{BUCKET}/{meta['model']}-{meta['name']}"
    if meta.get('cell_methods', False):
        base += f"-{meta['cell_methods']}"
    if  meta.get('height', False) and (len(meta['height'].strip().split(' ')) > 1):
        base += '-at_heights'
    if meta.get('pressure', False) and (len(meta['pressure'].strip().split(' ')) > 1):
        base += '-at_pressures'
    return base + '.zarr'
    

def zarr_store(meta):
    return OffSetS3Map(root=get_zar_path(meta), temp_chunk_path=meta['name'], check=False)
    
def msg_to_path(msg):
    return f'/s3/{msg["bucket"]}/{msg["key"]}'

def reshape_to_dest_cube(cube):
    return iris.util.new_axis(iris.util.new_axis(cube, 'forecast_period'), 'forecast_reference_time')


def get_proto_zarr_array(meta):
    OffSetS3Map(root=get_zar_path(meta), temp_chunk_path=meta['name'], check=False)
    array_store = OffSetS3Map(root=get_zar_path(meta) +'/' + meta['name'], temp_chunk_path='', check=False)
    return zarr.open(array_store)

def meta_from_zarr_path(path):
    decompose = path
    decompose = decompose.rsplit('.',1)[0]
    meta = {}
    
    models = ['mo-atmospheric-global-prd', 'mo-atmospheric-mogreps-g-prd', 'mo-atmospheric-ukv-prd', 'mo-atmospheric-mogreps-uk-prd']
    for possiable_model in models:
        if path.startswith(possiable_model):
            model = possiable_model
        
    meta['model'] = model
    
    decompose = decompose.replace(model+'-', '')
    
    if path.find('-at_heights'):
        height = "0 1 2 3"
        meta['height'] = height
        decompose = decompose.replace('-at_heights', '')
        
    if path.find('-at_pressures') > 0:
        pressure = "1000 2000 3000" 
        meta['pressure'] = pressure
        decompose = decompose.replace('-at_pressures', '')
        
    meta['name'] = decompose 
    
    return meta

In [2]:
# %load offsetmap.py
import s3fs
import re
import json


class OffSetS3Map(s3fs.S3Map):
    def __init__(self, *args, temp_chunk_path=None, **kwargs):
        self.temp_chunk_path = temp_chunk_path
        self._offsets = {}
        s3fs.S3Map.__init__(self, *args, **kwargs)
        self._chunk_re = re.compile("^[0-9]+([.][0-9]+)*$")
        
    def _get_offset(self, prefix):
        offset = self._offsets.get(prefix, False)
        if offset is False:
            offset = None
            try:
                path = prefix + '/.zattrs' if prefix else '.zattrs'
                attrs = super().__getitem__(path)
                offset = json.loads(attrs).get('_offset', None)
            except KeyError:
                print(f"key error {prefix}")
                pass
                
            self._offsets[prefix] = offset
        return offset
    
    def _is_chunk(self, path):
        return bool(self._chunk_re.match(path.split('/')[-1]))
    
    def __setitem__(self, path, item):
       
        if not self._is_chunk(path):
            return super().__setitem__(path, item)
        
        if not self.temp_chunk_path or not path.rsplit('/',1)[0].endswith(self.temp_chunk_path):
            return super().__setitem__(path, item)
        
        try: 
            self.s3.s3_additional_kwargs['Tagging'] = 'type=temp_chunk'
            return super().__setitem__(path, item)
        finally:
            del self.s3.s3_additional_kwargs['Tagging']
                
                
    def __getitem__(self, path):
        origpath = path
        if '/' in path:
            prefix, item = path.rsplit('/',1) 
        else:
            prefix = ''
            item = path
            
        if self._chunk_re.match(item):
            offset = self._get_offset(prefix)
            if offset:
                # Apply offset
                chunks = [int(i) for i in item.split('.')]
                ochunks = [chunk + offset[i] for i,chunk in enumerate(chunks)]
                item = '.'.join(str(v) for v in ochunks)
        
        path = prefix + '/' + item if prefix else item
        return super().__getitem__(path)

class PrintS3Map(s3fs.S3Map):
    def __getitem__(self, path):
        print(f"keep same {path} to {path}")
        return super().__getitem__(path)

## Find an example data sets to work with from the from the staging bucket manifest.
We use the staging bucket cause we can queriy it much beter 'cause there is a dynamo table



In [3]:
## Params
# These params pick a sizable data set
model="mo-atmospheric-mogreps-uk-prd"
param="air_temperature"
multi_height=True
cell_methods=None


In [4]:
manifest = intake.cat.service_hub_manifest.read()


Code below a little narly but basically it filters on staging bucket manifest based on the params above, this is `data_set`. It then finds which run in  `data_set` has the longest forecast period as not all runs are equal in this respect (major hours on uk-v run longer). This is what is finally represented as `one_run_data`. 

In [8]:
manifest_filter = (manifest.model == model) & (manifest.name == param) & (manifest.created_time.str.startswith('2019'))
if multi_height is True:
    manifest_filter &= (manifest.height.str.contains('5.0 10.0'))

if cell_methods:
    manifest_filter &= (manifest.cell_methods == cell_methods)
else:
    manifest_filter &= (manifest.cell_methods != manifest.cell_methods)
    

    
data_set = manifest[manifest_filter]

data_set.forecast_reference_time = pd.to_datetime(data_set.forecast_reference_time)
data_set.forecast_period = data_set.forecast_period.astype(float)
data_set = data_set.sort_values('forecast_period').sort_values('forecast_reference_time')
runs = sorted(data_set.forecast_reference_time.unique(), reverse=True)


runs = runs[1:]
runs

start = cur = longest_run = runs[0]
i = 0
longest_length = 0
for i, run in enumerate(runs):
    if (start - run) > np.timedelta64(timedelta(hours=6)):
        break
        
    run_length = len(data_set.forecast_period.unique())
    if run_length > longest_length:
        longest_length = run_length
        longest_run = run
    i+= 1

    
    
one_run_data = data_set[data_set.forecast_reference_time == longest_run]
one_run_data = one_run_data.sort_values('forecast_period')
one_run_data.head(3)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,bucket,cell_methods,created_time,depth,depth_units,forecast_period,forecast_period_bounds,forecast_period_units,forecast_reference_time,height,height_units,model,name,object_size,pressure,pressure_units,realization,time,ttl
key,,,,,,,,,,,,,,,,,,,
ed9b728d51491b993b77b04eadb43a2eaf770a62.nc,informatics-aws-earth-staging,NaN,2019-02-11T12:58:35Z,NaN,NaN,0.0,NaN,seconds,2019-02-11 09:00:00,5.0 10.0 20.0 30.0 50.0 75.0 100.0 150.0 200.0...,m,mo-atmospheric-mogreps-uk-prd,air_temperature,345797589,NaN,NaN,0 12 13 14 15 16 17 18 19 20 21 22,2019-02-11T09:00:00Z,1550132971
7599f8b74223bb31f2d3b6a862cd7ca14f29e26c.nc,informatics-aws-earth-staging,NaN,2019-02-11T12:58:35Z,NaN,NaN,3600.0,NaN,seconds,2019-02-11 09:00:00,5.0 10.0 20.0 30.0 50.0 75.0 100.0 150.0 200.0...,m,mo-atmospheric-mogreps-uk-prd,air_temperature,339661112,NaN,NaN,0 12 13 14 15 16 17 18 19 20 21 22,2019-02-11T10:00:00Z,1550131563
3b7e78618b581b882d36b5f11646be48a78314d4.nc,informatics-aws-earth-staging,NaN,2019-02-11T12:58:35Z,NaN,NaN,7200.0,NaN,seconds,2019-02-11 09:00:00,5.0 10.0 20.0 30.0 50.0 75.0 100.0 150.0 200.0...,m,mo-atmospheric-mogreps-uk-prd,air_temperature,337715255,NaN,NaN,0 12 13 14 15 16 17 18 19 20 21 22,2019-02-11T11:00:00Z,1550133303


In [10]:
forecast_periods = sorted([float(p) for p in one_run_data.forecast_period.unique()])
forecast_period_steps = set(forecast_periods[i+1] - forecast_periods[i] for i in range(len(forecast_periods) -1))
assert len(forecast_period_steps) ==1, f"Require constant forecast period step. Got {forecast_period_steps}"
forecast_period_step = forecast_period_steps.pop()
f"Forecast_period_step is {forecast_period_step} {one_run_data.iloc[0].forecast_period_units}"


'Forecast_period_step is 3600.0 seconds'

In [11]:
forecast_reference_times = sorted(data_set.forecast_reference_time.unique())
forecast_reference_time_steps = set([(forecast_reference_times[i+1] - forecast_reference_times[i] ) / np.timedelta64(1, 's') for i in range(len(forecast_reference_times)-1)])
assert len(forecast_reference_time_steps) == 1, f"Require constant forecast reference time steps. Got {forecast_period_steps} seconds"
forecast_reference_time_step = forecast_reference_time_steps.pop()

f"forecast_reference_time_step is {forecast_reference_time_step} secs"

'forecast_reference_time_step is 21600.0 secs'

In [8]:
# files = ['/s3/informatics-aws-earth-staging/' + key for key in data.index.get_level_values(0)]
# files[3:8]

['/s3/informatics-aws-earth-staging/13ec14adacd2f70e6593264654cb5db03d9a9323.nc',
 '/s3/informatics-aws-earth-staging/15a6543e28fa1e3ec1f5bb83aaec6e3f18addf8b.nc',
 '/s3/informatics-aws-earth-staging/1cec458cf3de4173a78bdc56ce5e79d7cb7bcd99.nc',
 '/s3/informatics-aws-earth-staging/21184f6392c57de71f85938caae86cadfac21e32.nc',
 '/s3/informatics-aws-earth-staging/25f059621e4e405b9cf0f735fab91640e43c8159.nc']

## Load proto cube and reshape
Reshape the proto cube such that it's the shape that we want for out final hyper-cube

In [ ]:
# cubes = iris.load(files)
# equalise_attributes(cubes)
# cube = cubes.concatenate_cube()
# cube.remove_coord('time')
# iris.util.promote_aux_coord_to_dim_coord(cube, 'forecast_period')
# proto_cube = iris.util.new_axis(cube, 'forecast_reference_time')
# proto_cube

In [29]:
# forecast_period = proto_cube.coord('forecast_period').points
# forecast_period_steps = set(forecast_period[i+1] - forecast_period[i] for i in range(len(forecast_period)-1))
# assert len(forecast_period_steps) == 1, f"Require constant forecast period step. Got {forecast_period_steps}"
# forecast_period_step = forecast_period_steps.pop()
# f"Forecast_period_step is {forecast_period_step}"

'Forecast_period_step is 3600'

In [12]:
msg = json.loads(one_run_data.iloc[0].to_json())
msg['key'] = one_run_data.index[0]

In [27]:
cube = iris.load_cube(msg_to_path(msg))
proto_cube = reshape_to_dest_cube(cube)
print(f"Run is {list(proto_cube.coord('forecast_reference_time').cells())[0].point}")
proto_cube

Run is 2019-02-11 09:00:00


<iris 'Cube' of air_temperature / (K) (forecast_reference_time: 1; forecast_period: 1; realization: 12; height: 33; projection_y_coordinate: 970; projection_x_coordinate: 1042)>

## Define chunks 
Chunk sizes must be factors of our proto cube shape so that any chunk is only writen to by one file.

In [125]:
def get_chunks(cube):
    chunks = []
    unit_chunk = ['forecast_reference_time','forecast_period']
    full_chunk = ['latitude','longitude','grid_latitude','grid_longitude','projection_y_coordinate','projection_x_coordinate']
    for i, coord in enumerate(cube.dim_coords):
        chunk = None
        if coord.name() in unit_chunk:
            chunk = 1

        elif coord.name() in full_chunk:
            chunk = proto_cube.shape[i]

        else:
            for factor in [11,7,5,3,2,1]:
                if proto_cube.shape[i] % factor == 0:
                    chunk = factor
                    break

        if chunk is None:
            raise ValueError(f"Don't know how to chunk coord {coord.name()}, shape {cube.shape[i]}")

        chunks.append(chunk)
    return chunks
        
    
    
chunks = get_chunks(proto_cube)
assert len(chunks) == len(proto_cube.shape)
assert sum([a%b for a,b in zip(proto_cube.shape, chunks)]) == 0 # No chunk should be in more than one file
print(f"chunks = {chunks}")

chunks = [1, 1, 3, 11, 970, 1042]


## Convert to xarray and save
Since xarray can nativly write zarr and Iris not lets convert.
We also need to rechunck the array to match our desired chunk format

In [144]:
xproto = xarray.DataArray.from_iris(proto_cube)
xproto_chunked = xproto.chunk(chunks)
ds = xproto_chunked.to_dataset()

In [145]:
print("Saving at s3://"+get_zar_path(msg))

Saving at s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr


In [146]:
# Create a cluster. Need to ensure it has write access to the write dest of the Zarr, hence using the template which has AWS keys in as Env vars
import dask_kubernetes
import distributed
dask_worker_template = os.path.expanduser("~/ota/small-worker-template.yaml")
cluster = dask_kubernetes.KubeCluster.from_yaml(dask_worker_template)
cluster.adapt(minimum=0,maximum=40)
c = distributed.Client(cluster)
cluster

In [147]:
!aws s3 rm --recursive s3://{get_zar_path(msg)}
compressor = Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE)
store = OffSetS3Map(root=get_zar_path(msg), temp_chunk_path=param, check=False)
encoding={proto_cube.name(): {'compressor': compressor}}

delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/.zattrs
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/0.0.0.0.0.0
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/.zarray
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/0.0.3.1.0.0
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/0.0.0.1.0.0
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/.zattrs
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/0.0.1.1.0.0
delete: s3://metoffice-aws-earth-zarr/mo-atmospheric-mogreps-uk-prd-air_temperature-at_heights.zarr/air_temperature/0.0.1.0.0.0
dele

In [148]:
ds.to_zarr(store=store, encoding=encoding)

In [ ]:
del c
del cluster
dask.config.set(scheduler=None)

## Proto cube complete
We now have our proto cube built
We should be able to load the data part as a zarr array


## Load the data array

In [150]:
z_arr = get_proto_zarr_array(msg)
z_arr

<zarr.core.Array (1, 1, 12, 33, 970, 1042) float32>

In [151]:
# Check chunking is as expeted
assert list(z_arr.chunks) == chunks

## Add metadata to allow index caluclation
When a new file comes in we need to be able to work out where in the n-d space it belongs. To do this we define the `origin` for the zarr array for each of the dims and we also define a step along that dim. If step is None then assume that the file is already at the origin for that dimention. We add the `origin` to the zarr attrs at `_origin`.
We write this info back into the zarr

* mo-atmospheric-mogreps-uk-prd: 3600 for step.Interval 6H at 15, 21, 03, 09
* mo-atmospheric-ukv-prd: 3600 for step. Interval 1H
* mo-atmospheric-mogreps-g-prd:10800 for step.  Interval 6H at 12,18,00,06
* mo-atmospheric-global-prd: {3600, 10800} for step. Interval 6H at 12,18,00,06

In [24]:
for c  in proto_cube.dim_coords:
    print( f'name {c.name()} unit {str(c.units)}')

name forecast_reference_time unit seconds since 1970-01-01 00:00:00
name forecast_period unit seconds
name realization unit 1
name height unit m
name projection_y_coordinate unit m
name projection_x_coordinate unit m


In [152]:
# if msg['model'] == 'mo-atmospheric-ukv-prd':
#     forecast_reference_time_step =  1*60*60
#     forecast_period = 3600
# elif msg['model'] == 'mo-atmospheric-mogreps-uk-prd':
#     forecast_reference_time_step =  6*60*60
#     forecast_period = 3600 
# elif msg['model'] == 'mo-atmospheric-mogreps-g-prd':
#     forecast_reference_time_step =  6*60*60
#     forecast_period = 10800
# else:
#     raise RuntimeError(f"Not sure about { msg['model']} since forecast_reference_time_step might not be const.")


dim_steps = {
    'forecast_reference_time': forecast_reference_time_step, 
    'forecast_period': forecast_period_step
}
origin = []
for c  in proto_cube.dim_coords:
    if c.name() == 'forecast_reference_time':
        assert 'seconds' in str(c.units).lower() # We've assumed that forecast_reference_time step is in seconds. Let's check.
    origin.append({
        'name': c.name(),
        'unit': str(c.units),
        'at': float(c.points[0]),
        'step': dim_steps.get(c.name(), None)
    })
origin

[{'name': 'forecast_reference_time',
  'unit': 'seconds since 1970-01-01 00:00:00',
  'at': 1547521200.0,
  'step': 21600.0},
 {'name': 'forecast_period', 'unit': 'seconds', 'at': 0.0, 'step': 3600.0},
 {'name': 'realization', 'unit': '1', 'at': 0.0, 'step': None},
 {'name': 'height', 'unit': 'm', 'at': 5.0, 'step': None},
 {'name': 'projection_y_coordinate',
  'unit': 'm',
  'at': -1036000.0,
  'step': None},
 {'name': 'projection_x_coordinate',
  'unit': 'm',
  'at': -1158000.0,
  'step': None}]

In [153]:
z_arr.attrs['_origin'] = origin
z_arr.attrs['_offset'] = [0 for i in range(len(z_arr.shape))]

In [154]:
z_arr.attrs['_offset']

[0, 0, 0, 0, 0, 0]

## Up date the meta data to be the shape of one complete run.

Update the forecast_period zarr so that it's the shape of a complete run. The dim def will be there but the underling data will not.

In [14]:
forecast_period_array = zarr.open(s3fs.S3Map(f's3://{get_zar_path(msg)}/forecast_period'))
assert str(forecast_period_array.dtype) == 'int32'
assert forecast_period_array.shape == (1,)
forecast_period_array.resize((len(forecast_periods,)))
forecast_period_array[:] = [int(p) for p in forecast_periods]
forecast_period_array.shape

AssertionError: 

Update the data zarr to the new shape now we've extended the forecast_period
Also update the shape of the data array to be one 24 hour rolling window.

In [37]:
assert msg['model'] == "mo-atmospheric-mogreps-uk-prd", f"Model must be 'mo-atmospheric-mogreps-uk-prd' got {msg['model']}. That model isn't implemented yet"

In [17]:
z_arr = get_proto_zarr_array(msg)
z_arr

<zarr.core.Array (1, 1, 12, 33, 970, 1042) float32>

In [40]:
24 // 6

4

In [41]:
forecast_period_index = -1
forecast_reference_time_index = -1

for i,c  in enumerate(proto_cube.dim_coords):
    if(c.name() == "forecast_period"):
        forecast_period_index = i
        
    if(c.name() == "forecast_reference_time"):
        forecast_reference_time_index = i
        
assert forecast_period_index >= 0, "Didn't find index for 'forecast_period' so can't update size"
assert forecast_reference_time_index >= 0, "Didn't find index for 'forecast_reference_time_index' so can't update size"


# Assume 24 hours rolling, one model every 6 hours there for 4 forecast_reference_time's


new_shape = list(z_arr.shape)
new_shape[forecast_period_index] = forecast_period_array.shape[0]
new_shape[forecast_reference_time_index] = 24 // 6

f"Old shape {proto_cube.shape} new shape {new_shape}"

'Old shape (1, 1, 12, 33, 970, 1042) new shape [4, 55, 12, 33, 970, 1042]'

In [42]:
z_arr.resize(new_shape)
z_arr.shape

(4, 55, 12, 33, 970, 1042)